In [2]:
from catboost import CatBoostRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score,confusion_matrix
import numpy as np
import pandas as pd

In [79]:
# train = pd.read_csv('C:/Users/doong/Desktop/공모전/운송량 예측/235867_물류 유통량 예측 경진대회/train_df.csv', encoding='cp949')
# test = pd.read_csv('C:/Users/doong/Desktop/공모전/운송량 예측/235867_물류 유통량 예측 경진대회/test_df.csv', encoding='cp949')
# ss = pd.read_csv('C:/Users/doong/Desktop/공모전/운송량 예측/235867_물류 유통량 예측 경진대회/sample_submission.csv', encoding='cp949')

In [3]:
train = pd.read_csv('C:/Users/doong/Desktop/PythonWorkspace/운송량예측공모전/train(smote).csv', encoding='utf-8')
test = pd.read_csv('C:/Users/doong/Desktop/PythonWorkspace/운송량예측공모전/test(전처리 끝).csv', encoding='utf-8')
ss = pd.read_csv('C:/Users/doong/Desktop/공모전/운송량 예측/235867_물류 유통량 예측 경진대회/sample_submission.csv', encoding='utf-8')

In [4]:
test.drop(["index"], axis=1, inplace=True)

In [6]:
train.drop(["index"], axis=1, inplace=True)

In [7]:
train.drop("Unnamed: 0", axis=1, inplace=True)

In [119]:
# train['DL_GD_LCLS_NM']=train['DL_GD_LCLS_NM'].astype('category')
# train['DL_GD_MCLS_NM']=train['DL_GD_MCLS_NM'].astype('category')

In [120]:
# # 특수문자(/) 제거
# train['DL_GD_LCLS_NM'] = ''.join(filter(str.isalnum, train['DL_GD_LCLS_NM'])) 
# train['DL_GD_MCLS_NM'] = ''.join(filter(str.isalnum, train['DL_GD_LCLS_NM'])) 

In [121]:
# # 대 + 중
# X = train.iloc[:,:-1]
# y = train["INVC_CONT"]  

In [8]:
# 중
X = train.drop("INVC_CONT", axis=1)
y = train["INVC_CONT"]  

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 2021)

In [125]:
y_train = np.log1p(y_train.values)

In [127]:
# cat_features = ['DL_GD_LCLS_NM', 'DL_GD_MCLS_NM']

In [128]:
# def objective(trial):
#     param = {}
#     param['learning_rate'] = trial.suggest_discrete_uniform("learning_rate", 0.001, 0.01, 0.005)
#     param['depth'] = trial.suggest_int('depth', 9, 15)
#     param['l2_leaf_reg'] = trial.suggest_discrete_uniform('l2_leaf_reg', 1.0, 5.5, 0.5)
#     param['min_child_samples'] = trial.suggest_categorical('min_child_samples', [8, 16, 32])
#     param['grow_policy'] = 'Depthwise'
#     param['iterations'] = 1000
#     param['use_best_model'] = True
#     param['eval_metric'] = 'RMSE'
#     param['od_type'] = 'iter'
#     param['od_wait'] = 20
#     param['random_state'] = 1
#     param['logging_level'] = 'Silent'
    
#     regressor = CatBoostRegressor(**param)

#     regressor.fit(X_train, y_train,
#                   cat_features=cat_features,
#                   eval_set=[(X_val, y_val.copy())],
#                   early_stopping_rounds=100)
#     loss = np.sqrt(mean_squared_error(X_val, regressor.predict(X_val)))
#     return loss

In [129]:
# import optuna
# from optuna.samplers import TPESampler

# study = optuna.create_study(study_name=f'catboost-seed{1}')
# study.optimize(objective, n_trials=5)
# print("Best Score:",study.best_value)
# print("Best trial",study.best_trial.params)

In [130]:
# study.best_value

In [131]:
# study.best_params

In [10]:
cat = CatBoostRegressor(
    iterations = 500,
    learning_rate = 0.001,
    depth = 10,
    eval_metric = 'RMSE',
    random_seed = 2021,
    bagging_temperature = 0.2,
    od_type = 'Iter',
    metric_period = 50,
    od_wait = 20,
    l2_leaf_reg = 3,
    min_child_samples= 16,
    use_best_model=True
)

In [12]:
cat.fit(X_train, y_train,
          eval_set=[(X_train, y_train), (X_test,y_test)],
          early_stopping_rounds=35,
          verbose=100)

0:	learn: 8.2541153	test: 8.2541153	test1: 8.2746620	best: 8.2746620 (0)	total: 404ms	remaining: 3m 21s
100:	learn: 8.1311472	test: 8.1311472	test1: 8.1560435	best: 8.1560435 (100)	total: 12.9s	remaining: 51.1s
200:	learn: 8.0262567	test: 8.0262567	test1: 8.0556185	best: 8.0556185 (200)	total: 29.7s	remaining: 44.2s


KeyboardInterrupt: 

In [134]:
test_X = test.iloc[:,:]
pred = cat.predict(test_X)
pred = np.exp(pred)
ss["INVC_CONT"] = pred

In [135]:
ss

,index,INVC_CONT
0,32000,5.132176
1,32001,5.132176
2,32002,5.132176
3,32003,5.132176
4,32004,5.132176
...,...,...
4635,36635,5.132176
4636,36636,5.132176
4637,36637,5.132176
4638,36638,5.132176


In [136]:
ss.to_csv('sample_submssion.csv',index = False)